# Example usage on HPE

### 1. Connect to the spark session

### 2. Download the source code by the procedure of [README.md](../README.md)

In [11]:
%%bash

svn checkout https://github.com/HinnyTsang/pysparkchannel/trunk/pysparkchannel

Checked out revision 8.


### 3. Run the following cells

The example will regenerate `module_a` and `module_b` in `dest` directory.

In [22]:
# %%local
# Run this cell in local

# Import the pysparkchannel module
import os
import shutil
import sys
sys.path.append('..')
import importlib

from pysparkchannel import core
importlib.reload(core)

# Remove file if exist
if os.path.exists("dest"):
    shutil.rmtree("dest", ignore_errors=True)

# Create the parser
parser = core.ModuleParser(verbose = False)

# Parse the cus
# tom module one by one.
parser \
    .parse_module("module_a") \
    .parse_module("module_b")

# Generate script to rewrite the module on spark cluster
script = parser.generate_script("dest")

In [24]:
# %%spark
import json
import subprocess

parsed_modules = parser.parsed_modules \
    .replace('`', '"')

script = script.replace('`', '"')

with open("structure.json", "w", encoding="utf-8") as file:
    json.dump(parsed_modules, file, ensure_ascii=True, indent=4)
    
with open("script.py", "w", encoding="utf-8") as file:
    file.write(script)

subprocess.check_output(["python", "script.py"])    

Traceback (most recent call last):
  File "/home/hinny/data-science/pysparkchannel/example/script.py", line 66, in <module>
    main()
  File "/home/hinny/data-science/pysparkchannel/example/script.py", line 62, in main
    reconstruct_modules(decode_expression(BASE_PATH))
  File "/home/hinny/data-science/pysparkchannel/example/script.py", line 29, in reconstruct_modules
    loaded_modules: Dict[str, Folder] = json.loads(decode_expression(JSON_STRING))
  File "/home/hinny/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/home/hinny/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/hinny/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


CalledProcessError: Command '['python', 'script.py']' returned non-zero exit status 1.

### 4. Use spark magic to copy the script to the spark session

In [4]:
# %%send_to_spark -i script -t str -n script

### 5. Run the script on the spark session to regenerate the modules

In [5]:
# %%spark

# Execute the script to rebuild the modules
exec(script)

Reconstructing folder dest/module_a
Reconstructing file dest/module_a/__init__.py
Reconstructing folder dest/module_b
Reconstructing folder dest/module_b/sub_module_i
Reconstructing file dest/module_b/sub_module_i/sub_module_ii.py
Reconstructing file dest/module_b/sub_module_i/__init__.py
Reconstructing file dest/module_b/module_bi.py
Reconstructing file dest/module_b/__init__.py
